# Tải các package cần thiết

In [1]:
!pip install --quiet -U langchain chromadb langchain-openai pypdf gradio datasets langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.3 MB/s eta 0

In [17]:
# Tải groq
!pip install -q groq

# Load và Split documents

In [6]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=50)
loader = PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/Buổi 7/vinallama.pdf")
splits = loader.load_and_split(text_splitter)

# Kiểm tra các Documents được tạo ra

In [9]:
splits[20].page_content

'performance assessment. This methodology employs an ELO ranking system, traditionally used in\nchess and other competitive games, to rate the models. Such a system offers a dynamic and rela-\ntive measure of model performance, allowing for a nuanced and comparative analysis of each model’s\nproficiency in handling a variety of tasks and instructions within the benchmark. This ELO-based\nevaluation provides a clear and quantifiable ranking of the models, reflecting their effectiveness and\nadaptability in the context of the Vietnamese language and the specific challenges presented by the\nVicuna Benchmark.\nIn our Vicuna Benchmark evaluation, responses from models were assessed using a detailed five-\npoint scale: 0 (’very bad’), 1 (’bad’), 2 (’ok’), 3 (’good’), and 4 (’very good’). This granular scoring\nsystem allows for an in-depth evaluation of the quality of each model’s response. The final ELO score\nfor each model is computed by aggregating these individual ratings, providing a 

# Tạo prompt cho synthetic data

In [10]:
synthetic_prompt = """
Bạn sẽ được cung cấp với một đoạn văn bản. Nhiệm vụ của bạn là hãy dựa trên nội dung này và tạo ra 20 cặp câu hỏi theo dạng FAQ mà một người bình thường sẽ hỏi:
---CONTEXT---
{context}
---END---
Bạn hãy tạo ra 10 cặp câu hỏi bằng Tiếng Việt theo dạng FAQ mà một người bình thường sẽ hỏi với format như sau:
---FORMAT INSTRUCTION---
Question:
<question1>
Answer:
<answer1>

Question:
<question2>
Answer:
<answer2>
... do this 10 times
---END---
Now, let's start
---START---
"""

In [11]:
splits[0].page_content

'VinaLLaMA: LLaMA-based Vietnamese Foundation Model\nQuan Nguyen∗, Huy Pham and†Dung Dao‡\nDecember 19, 2023\nAbstract\nIn this technical report, we present VinaLLaMA, an open-weight, state-of-the-art (SOTA)\nLarge Language Model for the Vietnamese language, built upon LLaMA-2 with an additional\n800 billion trained tokens. VinaLLaMA not only demonstrates fluency in Vietnamese but also\nexhibits a profound understanding of Vietnamese culture, making it a truly indigenous model.\nVinaLLaMA-7B-chat, trained on 1 million high-quality synthetic samples, achieves SOTA results\non key benchmarks, including VLSP, VMLU, and Vicuna Benchmark Vietnamese, marking a\nsignificant advancement in the Vietnamese AI landscape and offering a versatile resource for various\napplications.\n1 Introduction\nThe surge in Large Language Models (LLMs) such as ChatGPT and GPT-4 has significantly advanced\nthe field of artificial intelligence (AI), particularly in language processing. In 2023, Vietnam’s AI'

# Đẩy dữ liệu từ documents split lên context của synthetic prompt

In [13]:
gen_prompt = synthetic_prompt.format(context=splits[0].page_content, context2=splits[1].page_content)

In [14]:
print(gen_prompt)


Bạn sẽ được cung cấp với một đoạn văn bản. Nhiệm vụ của bạn là hãy dựa trên nội dung này và tạo ra 20 cặp câu hỏi theo dạng FAQ mà một người bình thường sẽ hỏi:
---CONTEXT---
VinaLLaMA: LLaMA-based Vietnamese Foundation Model
Quan Nguyen∗, Huy Pham and†Dung Dao‡
December 19, 2023
Abstract
In this technical report, we present VinaLLaMA, an open-weight, state-of-the-art (SOTA)
Large Language Model for the Vietnamese language, built upon LLaMA-2 with an additional
800 billion trained tokens. VinaLLaMA not only demonstrates fluency in Vietnamese but also
exhibits a profound understanding of Vietnamese culture, making it a truly indigenous model.
VinaLLaMA-7B-chat, trained on 1 million high-quality synthetic samples, achieves SOTA results
on key benchmarks, including VLSP, VMLU, and Vicuna Benchmark Vietnamese, marking a
significant advancement in the Vietnamese AI landscape and offering a versatile resource for various
applications.
1 Introduction
The surge in Large Language Models (LLMs)

# Khởi tạo GROQ

In [19]:
from groq import Groq
from google.colab import userdata

userdata.get('GROQ_API_KEY')
llm = Groq(api_key=userdata.get('GROQ_API_KEY'))


# Tạo synthetic data từ GROQ

In [20]:
list_qa_question = []
for split in splits[10:20]:
  gen_prompt = synthetic_prompt.format(context=split.page_content)
  response = llm.chat.completions.create(model='llama3-70b-8192',
              messages=[{"role": "user", "content": gen_prompt}],
              temperature=0.1,)

  list_qa_question.append(response)

In [21]:
len(list_qa_question)

10

In [29]:
list_qa_question[1].choices[0].message

ChatCompletionMessage(content='Here are 10 FAQ pairs in Vietnamese based on the provided context:\n\n---FAQ---\n\nQuestion:\nMô hình ngôn ngữ nào đã được đào tạo trong bốn kỷ nguyên?\nAnswer:\nMột mô hình ngôn ngữ cơ sở tiên tiến cho ngôn ngữ Việt Nam đã được đào tạo trong bốn kỷ nguyên.\n\nQuestion:\nMô hình VinaLLaMA-2.7B là gì?\nAnswer:\nVinaLLaMA-2.7B là một biến thể mô hình nhỏ hơn với 2,7 tỷ tham số, được tạo ra bằng cách áp dụng quy trình cắt tỉa cấu trúc cho mô hình gốc.\n\nQuestion:\nTại sao lại cần cắt tỉa mô hình?\nAnswer:\nCắt tỉa mô hình giúp giảm kích thước mô hình mà vẫn giữ lại các chức năng cốt lõi của nó.\n\nQuestion:\nPhương pháp đào tạo nào đã được sử dụng cho mô hình 2.7B?\nAnswer:\nPhương pháp đào tạo giám sát giống nhau đã được sử dụng cho mô hình 2.7B và mô hình 7B.\n\nQuestion:\nThời gian đào tạo mô hình là bao lâu?\nAnswer:\nGiai đoạn đào tạo trước đã được hoàn thành trong một tuần, trong khi giai đoạn đào tạo tinh chỉnh đã được hoàn thành nhanh hơn.\n\nQuesti

# Format lại câu hỏi và câu trả lời

In [38]:
text = '''
Question:\nMô hình ngôn ngữ nào đã được đào tạo trong bốn kỷ nguyên?\nAnswer:\nMột mô hình ngôn ngữ cơ sở tiên tiến cho ngôn ngữ Việt Nam đã được đào tạo trong bốn kỷ nguyên.
'''
text = text.split('Question:\n')[1:]
for sentence in text:
  question_part, answer_part = sentence.split('\nAnswer:\n')
  print(f'Question: {question_part}')
  print(f'Answer:  {answer_part}')

Question: Mô hình ngôn ngữ nào đã được đào tạo trong bốn kỷ nguyên?
Answer:  Một mô hình ngôn ngữ cơ sở tiên tiến cho ngôn ngữ Việt Nam đã được đào tạo trong bốn kỷ nguyên.



In [39]:
def parse_qa_pairs(text):
    """
    Parses a given text containing questions and answers into a list of dictionaries.

    Parameters:
    - text (str): A string containing questions and answers in a structured format.

    Returns:
    - List[Dict[str, str]]: A list of dictionaries, each representing a question-answer pair.
    """
    qa_pairs_simplified = []

    # Split the text based on "Question:" as a delimiter and ignore the first split which is empty
    sections = text.split("Question:\n")[1:]

    for section in sections:
        # Each section contains one question and one answer split by "Answer:"
        question_part, answer_part = section.split("\nAnswer:\n")
        question = question_part.strip()
        answer = answer_part.strip()
        qa_pairs_simplified.append({"question": question, "answer": answer})

    return qa_pairs_simplified

In [41]:
qa = []
for qa_pair in list_qa_question:
  pair = qa_pair.choices[0].message.content
  qa.append(parse_qa_pairs(pair))

In [43]:
print(qa[0])

[{'question': 'Mục đích của giai đoạn thứ hai trong việc tạo dữ liệu tổng hợp là gì?', 'answer': 'Mục đích của giai đoạn thứ hai là để tiết kiệm chi phí trong khi vẫn duy trì hiệu suất giống nhau với quá trình chưng cất từ GPT-4.'}, {'question': 'Vietcuna-3B-v2 được sử dụng để làm gì?', 'answer': 'Vietcuna-3B-v2 được triển khai để tiết kiệm chi phí trong khi vẫn duy trì hiệu suất giống nhau với quá trình chưng cất từ GPT-4.'}, {'question': 'Tại sao lại cần có 500,000 mẫu tiếng Việt tổng hợp?', 'answer': '500,000 mẫu tiếng Việt tổng hợp được tạo ra để tăng cường dữ liệu và phát triển mô hình ngôn ngữ song ngữ.'}, {'question': 'Ngoài các mẫu tiếng Việt, còn có thêm các mẫu tiếng Anh nào khác?', 'answer': 'Ngoài các mẫu tiếng Việt, còn có thêm 500,000 mẫu tiếng Anh được lấy từ các bộ dữ liệu OpenHermes-2.5 và Capybara.'}, {'question': 'Tại sao lại cần có các mẫu tiếng Anh?', 'answer': 'Các mẫu tiếng Anh được thêm vào để tăng cường dữ liệu và phát triển mô hình ngôn ngữ song ngữ.'}, {'ques

# Chuyển định dạng về dạng ShareGPT để finetuning

In [44]:
sharegpt_data = []
for group in qa:
  for pair in group:
    convo = {}
    convo['conversations'] = [{"from": "human", "value": pair['question']}, {"from": "gpt", "value": pair['answer']}]
    sharegpt_data.append(convo)

In [45]:
sharegpt_data[0]

{'conversations': [{'from': 'human',
   'value': 'Mục đích của giai đoạn thứ hai trong việc tạo dữ liệu tổng hợp là gì?'},
  {'from': 'gpt',
   'value': 'Mục đích của giai đoạn thứ hai là để tiết kiệm chi phí trong khi vẫn duy trì hiệu suất giống nhau với quá trình chưng cất từ GPT-4.'}]}

In [46]:
# Lưu lại file sharegpt

import json

with open('sharegpt_data.json', 'w', encoding='utf-8') as f:
  json.dump(sharegpt_data, f, indent=4, ensure_ascii=False)

In [47]:
import datasets

dataset = datasets.load_dataset('json', data_files='sharegpt_data.json')

Generating train split: 0 examples [00:00, ? examples/s]